In [1]:
# Dataset: 10.5281/zenodo.1048301
from marsvision.utilities import DataUtility

In [2]:
dataset_root = "X:\hirise-map-proj"

In [3]:
import os
import cv2
from torch.utils.data import Dataset
from torchvision import transforms
from torch import Tensor
class DeepMarsData(Dataset):
    # Wrapper to work with deep mars dataset
    def __init__(self, root_dir):
        
        # AlexNet expects images to be normalized this way.
        self.normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        )
        # Get image labels
        self.labels = {}
        with open(os.path.join(root_dir, "labels-map-proj.txt")) as f:
            for line in f:
                items = line.split()
                key, value = items[0], items[1]
                self.labels[key] = int(value)
                
        # Get image filenames
        self.image_dir = os.path.join(root_dir, "map-proj")
        image_names = os.listdir(os.path.join(self.image_dir))
        # Take set difference 
        # to ensure that only labelled images are included
        self.image_names = list(set(image_names) & set(self.labels))
                                  
    def __getitem__(self, idx):
        # Get a sample as: {'image': image, 'label': label}
        # Return an image with the dimensions 3 x W x H
        # Because PyTorch models expect these dimensions as inputs.
        # Transpose dimensions:
        # (W, H, 3) --> (3, W, H)
        img_name = self.image_names[idx]
        img = Tensor(
            cv2.imread(os.path.join(self.image_dir, img_name))
        ).transpose(0, 2)
        
        # Apply normalize
        img = self.normalize(img)
    
        return {
            "image": img,
            "label": self.labels[self.image_names[idx]]
        }
        
    def __len__(self):
        return len(self.image_names)

In [4]:
import torch
from torch.utils.data import random_split

# Splt into train/validation/test sets
# Train/Val/Test: 80/5/15
dataset = DeepMarsData(dataset_root)

# Define DataUtility for the whole dataset
dataset_size = len(dataset)
num_train_samples = int(dataset_size * .8)
num_val_samples = int(dataset_size * .05)
num_test_samples = dataset_size - num_train_samples - num_val_samples
data_sizes = {
    "train": num_train_samples,
    "val": num_val_samples,
    "test": num_test_samples
}

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset, 
    [num_train_samples, 
     num_val_samples, 
     num_test_samples]
)

dataloaders = {
    "train": torch.utils.data.DataLoader(train_dataset, batch_size = 4),
    "val": torch.utils.data.DataLoader(train_dataset, batch_size = 4),
    "test": torch.utils.data.DataLoader(train_dataset, batch_size = 4)
}


# Define a tiny datset to make quick tweaks to the model training code
# Use a small subset of the dataset
# So we can train it quickly.
dataset_smaller, dataset_larger = torch.utils.data.random_split(
    dataset,
    [ 100, dataset_size - 100 ]
)

dataset_smaller_size = len(dataset_smaller)
num_train_samples_smaller = int(dataset_smaller_size * .8)
num_val_samples_smaller = int(dataset_smaller_size * .05)
num_test_samples_smaller = dataset_smaller_size - num_train_samples_smaller - num_val_samples_smaller
train_dataset_smaller, val_dataset_smaller, test_dataset_smaller = torch.utils.data.random_split(
    dataset_smaller, 
    [num_train_samples_smaller, 
     num_val_samples_smaller, 
     num_test_samples_smaller]
)

data_sizes_smaller = {
    "train": num_train_samples_smaller,
    "val": num_val_samples_smaller,
    "test": num_test_samples_smaller
}

dataloaders_smaller = {
        "train": torch.utils.data.DataLoader(train_dataset_smaller, batch_size = 4),
        "val": torch.utils.data.DataLoader(val_dataset_smaller, batch_size = 4),
        "test": torch.utils.data.DataLoader(test_dataset_smaller, batch_size = 4)
}


In [5]:
# Initialize Alexnet and print its architecture.
# Notice the last layer of the classifier.
model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)
model

Using cache found in C:\Users\dpale/.cache\torch\hub\pytorch_vision_v0.6.0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [14]:
# Playing with the model/ Making predictions
sample = next(iter(dataloaders["train"])) 
output = model(sample["image"]) # Output tensor of shape: (samples, # of classes)
values, indices = torch.max(output, 1) # Get max values of confidence scores output by AlexNet.
# Indices = classes
# Use torch.max to return class label with highest confidence score.
output[0][indices[0]], values, indices

(tensor(274.1620, grad_fn=<SelectBackward>),
 tensor([274.1620, 476.5874, 478.7771, 693.4528], grad_fn=<MaxBackward0>),
 tensor([111, 530, 530, 530]))

In [15]:
from torch import nn
# To change the number of output features, modify the classifier like so.
# Classes correspond to indices.
num_classes = 7
model.classifier[6] = nn.Linear(4096,num_classes)
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
# Defining a function to train the model
def train_model(dataloaders, data_sizes, model, criterion, optimizer, scheduler, num_epochs=25):
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs - 1))
        print("-" * 10)
        
        # Each epoch has a training and validation phase
        for phase in ["train", "val"]:
            # Swap training/eval modes depending on phase
            if phase == "train":
                model.train()
            else:
                # Eval switches model's behavior
                # Enable eval when we need to evaluate the model.
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            # Iterate and train.
            for sample in DataUtilitys[phase]:
                inputs = Tensor(sample["image"]).to(device)
                labels = sample["label"]
                
                # Zero the gradients
                optimizer.zero_grad()
                
                # Forward pass if in train phase
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels)
                
                print("Running loss: {} | Running corrects: {}".format(
                    running_loss, running_corrects))
                
                if phase == 'train':
                    scheduler.step()
                    
            epoch_loss = running_loss / data_sizes[phase]
            epoch_acc = running_corrects.double() / data_sizes[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f} | Images trained on: {}'.format(
                phase, epoch_loss, epoch_acc, data_sizes[phase]))
            
            # In the eval phase, get the accuracy for this epoch
            # If the mode's current state is better than the best model seen so far,
            # replace the best model weights
            # with the previous best model weights on previous epochs
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# Run this cell to train AlexNet on the small dataset
# Don't expect accuracy to be very good because the dataset used is tiny
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import copy
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=1, gamma=0.1)
model_transfer = train_model(
    DataUtilitys_smaller, 
    data_sizes_smaller,
    model, 
    criterion, 
    optimizer_ft, 
    exp_lr_scheduler,
    num_epochs=4
)

In [ ]:
# Show an example of crossentropyloss.
# Crossentropyloss expects "raw, unnormalized scores" for each class.
sample = next(iter(DataUtilitys["train"])) 
scores = model(sample["image"])
scores, criterion(scores, sample["label"]).item()